In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 2.7MB/s 
     |████████████████████████████████| 8.2MB 7.0MB/s 


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import csv
from nltk.stem import WordNetLemmatizer
import sklearn
import codecs
import pymorphy2
import seaborn as sns
sns.set_style("darkgrid")
from nltk.stem.snowball import SnowballStemmer

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

###Функции

In [45]:
from nltk.corpus import stopwords
stopWords = set(stopwords.words('russian'))

def csv_to_list(arr):
    arr_list = []
    for row in arr:
        arr_list.append(list_to_str(row))
    return arr_list

def list_to_str(arr):
    str_ = ''
    for rec in arr:
        str_+=rec
    return str_

def df_preprocess(text):  
    reg = re.compile('[^а-яА-яa-zA-Z0-9 ]') #
    text = text.lower().replace("ё", "е")
    text = text.replace("ъ", "ь")
    text = text.replace("й", "и")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'сайт', text)
    text = re.sub('@[^\s]+', 'пользователь', text)
    text = reg.sub(' ', text)
    
    # Лемматизация
    #morph = pymorphy2.MorphAnalyzer()
    #text =[morph.parse(word)[0].normal_form for word in text.split()]

    # Стемминг
    # stemmer = SnowballStemmer("russian")
    # text =[stemmer.stem(word) for word in text.split()]

    # Стемминг + удаление стоп слов
    stemmer = SnowballStemmer("russian")
    #text =[stemmer.stem(word) for word in text.split() if word not in stopWords]
    text = ' '.join([stemmer.stem(word) for word in text.split() if word not in stopWords])

    return text

###Загрузка датасета

In [35]:
df_positive_recalls = pd.read_csv('drive/MyDrive/dataset/positive.csv', 
                               header=None,
                               sep=';',
                               error_bad_lines=False,
                               warn_bad_lines=False)

In [36]:
df_positive_recalls['recall'] = df_positive_recalls.iloc[:,3]
df_positive_recalls['type'] = 1
df_positive_recalls = df_positive_recalls.loc[:,['recall','type']]

In [37]:
df_positive_recalls.head()

,recall,type
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [38]:
df_negative_recalls = pd.read_csv('drive/MyDrive/dataset/negative.csv', 
                               header=None,
                               sep=';',
                               error_bad_lines=False,
                               warn_bad_lines=False)

In [39]:
df_negative_recalls['recall'] = df_negative_recalls.iloc[:,3]
df_negative_recalls['type'] = 0
df_negative_recalls = df_negative_recalls.loc[:,['recall','type']]

In [40]:
df_negative_recalls.head()

,recall,type
0,на работе был полный пиддес :| и так каждое за...,0
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",0
2,@elina_4post как говорят обещаного три года жд...,0
3,"Желаю хорошего полёта и удачной посадки,я буду...",0
4,"Обновил за каким-то лешим surf, теперь не рабо...",0


In [41]:
# Объединяем два датафрейма вместе и перемешиваем
df_recalls = pd.concat((df_negative_recalls, df_positive_recalls),axis = 0).sample(frac = 1.0) # объединяем и перемешиваем

In [44]:
# Обновляем индекс
df_recalls.reset_index(drop=True)

,recall,type
0,"даже в самое плохое настроение,я ржу над этим ...",1
1,@Apocalypse1boom ну их мало( http://t.co/hX0ad...,0
2,RT @roxusubegal: 31 января мы дружно шагаем в ...,1
3,"Ахах, мне доверили роль Пеппи!) Главное, тепер...",1
4,RT @shipilavaa: @1D_London_ILY это я слышу от ...,1
...,...,...
226829,"@MisterSmolders ахххах, нет. снилась таня из у...",1
226830,@ohh_Jula их размеры не сравняться с железнодо...,1
226831,"Мама разбудила со словами: ""вставай давай, а т...",0
226832,@Julijaz1 а я ваще ничо не писал еще (\nстыыд....,0


###Очитска текста приведение слов к стандартному виду

In [46]:
%time df_recalls['recall'] = df_recalls['recall'].apply(df_preprocess)

CPU times: user 1min 51s, sys: 189 ms, total: 1min 51s
Wall time: 1min 51s


In [47]:
df_recalls.head()

,recall,type
82089,сам плох настроен ржу эт виде d сайт провер 28...,1
79164,пользовател мал сайт,0
109335,rt пользовател 31 январ дружн шага мпк громад,1
55051,ах довер рол пепп главн выуч слов,1
33167,rt пользовател пользовател эт слыш само мило д...,1


###Train/test split

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_recalls['recall'], df_recalls['type'], test_size=.15, random_state=42)

In [49]:
print(len(X_train), len(X_test))

192808 34026


###BOW

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

X_train_BOW = vectorizer.fit_transform(X_train)
X_test_BOW = vectorizer.transform(X_test)

In [51]:
print(X_train_BOW.shape, X_test_BOW.shape)

(192808, 93649) (34026, 93649)


###TF-IDF

In [52]:
from sklearn.feature_extraction.text import  TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_TFIDF = vectorizer.fit_transform(X_train)
X_test_TFIDF = vectorizer.transform(X_test)

In [53]:
print(X_train_TFIDF.shape, X_test_TFIDF.shape)

(192808, 93649) (34026, 93649)


###Модель LogisticRegression

####BOW

In [54]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [55]:
# обучаем классификатор
%time clf = LogisticRegression(random_state=0).fit(X_train_BOW, y_train)

CPU times: user 7.04 s, sys: 8.27 s, total: 15.3 s
Wall time: 7.93 s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [56]:
# вычисляем предсказания
y_predict_BOW = clf.predict(X_test_BOW)

In [57]:
# вычисляем метрику accuracy
accuracy_score(y_predict_BOW, y_test)

0.734085699171222

####TF-IDF

In [58]:
%time clf = LogisticRegression(random_state=43).fit(X_train_TFIDF, y_train)

CPU times: user 6.9 s, sys: 7.4 s, total: 14.3 s
Wall time: 7.36 s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [59]:
y_predict_TFIDF = clf.predict(X_test_TFIDF)

In [60]:
accuracy_score(y_predict_TFIDF, y_test)

0.7344089813671898

###На данных BOW с биграммами

In [61]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
X_train_BOW_bi = vectorizer.fit_transform(X_train)
X_test_BOW_bi = vectorizer.transform(X_test)
print(X_train_BOW_bi.shape, X_test_BOW_bi.shape)

clf = LogisticRegression(random_state=0).fit(X_train_BOW_bi, y_train)
y_predict_BOW_bi = clf.predict(X_test_BOW_bi)
accuracy_score(y_predict_BOW_bi, y_test)

(192808, 867295) (34026, 867295)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7538646917063422

###Подбор гиперпараметров / отбор фичей / grid search / cross-validation

###Отбор фичей

In [62]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

lsvc = LinearSVC(C = .5) # C = 0.5
selective_model = SelectFromModel(lsvc,  max_features  = None)

X_train_BOW_bi_select_features = selective_model.fit_transform(X_train_BOW_bi,y_train)
X_test_BOW_bi_select_features = selective_model.transform(X_test_BOW_bi)
print('\nNew shapes: ', X_train_BOW_bi.shape, X_test_BOW_bi.shape)
print('\nNew shapes: ', X_train_BOW_bi_select_features.shape, X_test_BOW_bi_select_features.shape)


New shapes:  (192808, 867295) (34026, 867295)

New shapes:  (192808, 349950) (34026, 349950)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [63]:
clf = LogisticRegression(random_state=0)
scores = cross_val_score(clf, X_train_BOW_bi_select_features, y_train, cv=3, scoring='accuracy')
print(clf,'\n Cross-validate: ', scores)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False) 
 Cross-validate:  [0.7718998  0.77371672 0.77549052]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [64]:
clf = LogisticRegression(random_state=0).fit(X_train_BOW_bi_select_features, y_train)

y_predict_BOW_bi = clf.predict(X_test_BOW_bi_select_features)

accuracy_score(y_predict_BOW_bi, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7496326338682184

###GridSearch

In [65]:
from sklearn.model_selection import GridSearchCV

In [66]:
grid={
      "C":np.logspace(-3,3,4), 
      "penalty":["l2"],
      "solver":['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
      }

clf=LogisticRegression()
logreg_cv=GridSearchCV(clf,grid,cv=3)
%time logreg_cv.fit(X_train_BOW_bi_select_features, y_train)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


CPU times: user 11min 51s, sys: 7min 32s, total: 19min 24s
Wall time: 11min 31s
tuned hpyerparameters :(best parameters)  {'C': 10.0, 'penalty': 'l2', 'solver': 'liblinear'}
accuracy : 0.7881260302410228


In [67]:
logreg_cv.best_params_

{'C': 10.0, 'penalty': 'l2', 'solver': 'liblinear'}

In [68]:
clf = LogisticRegression(C=10.0, penalty='l2', solver='liblinear').fit(X_train_BOW_bi_select_features, y_train)

y_predict_BOW_bi = clf.predict(X_test_BOW_bi_select_features)

accuracy_score(y_predict_BOW_bi, y_test)

0.7462234761652854